# Quantum Computing for the time-varying Linear Quadratic Regulator

This notebook implements the example outlined in the `QuantumLQR.tex` file. We will follow the steps described in sections 4, 5, and 6 to solve the matrix Riccati equation and find the control function.

## Import necessary libraries

We will start by importing the required libraries such as `numpy`, `qiskit`, and any other necessary modules from the repository.

In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.circuit.library import HamiltonianGate, PhaseEstimation, StatePreparation
from qiskit.quantum_info import Statevector
from enhanced_hybrid_hhl import HHL, Lee_preprocessing, EnhancedHybridInversion, QuantumLinearSystemProblem, QuantumLinearSystemSolver

## Define the problem

We will define the time-varying Linear Quadratic Regulator (LQR) problem and the associated matrix Riccati equation as described in the `QuantumLQR.tex` file.

In [2]:
# Define the matrices and parameters for the LQR problem
A_matrix = np.array([[1, 0], [0, 1]])
B_matrix = np.array([[np.exp(0.5), -np.exp(0.5)], [np.exp(0.5), np.exp(0.5)]])
Q_matrix = np.array([[0.5 * np.exp(-t), 0], [0, 0.5 * np.exp(-t)]])
R_matrix = np.identity(2)
t_f = 1
t_0 = 0
x_0 = np.array([1, 0])
x_f = np.array([0, 0])

## Change of variable

We will implement the change of variable to transform the matrix Riccati equation into a second-order matrix linear differential equation.

In [3]:
# Define the change of variable
def change_of_variable(A, B, Q, R, t_f):
    S = 2 * np.identity(2) + np.gradient(B @ B.T) @ np.linalg.inv(B @ B.T)
    V = np.zeros((2, 2))
    V_prime = np.zeros((2, 2))
    V_double_prime = np.zeros((2, 2))
    for t in np.linspace(t_0, t_f, 100):
        V_double_prime = S @ V_prime + B @ B.T @ Q @ V
        V_prime += V_double_prime * (t_f - t_0) / 100
        V += V_prime * (t_f - t_0) / 100
    return V

## Solve the differential equation

We will use the HHL algorithm to solve the matrix equations equivalent to the set of linear differential equations.

In [4]:
# Define the HHL algorithm parameters
k = 3  # clock qubits for HHL
l = k + 2  # clock qubits for enhanced preprocessing
max_eigenvalue = 1  # Overestimate of largest eigenvalue in the system
simulator = Aer.get_backend('qasm_simulator')

# Define the preprocessing function
enhanced_preprocessing = Lee_preprocessing(num_eval_qubits=l, max_eigenvalue=max_eigenvalue, backend=simulator).estimate
enhanced_eigenvalue_inversion = EnhancedHybridInversion

# Create the HHL class
enhanced_hybrid_hhl = HHL(get_result_function='get_swap_test_result', preprocessing=enhanced_preprocessing, eigenvalue_inversion=enhanced_eigenvalue_inversion, backend=simulator)

# Define the quantum linear system problem
problem = QuantumLinearSystemProblem(A_matrix=A_matrix, b_vector=x_0)

# Run the HHL algorithm
enhanced_hybrid_hhl_result = enhanced_hybrid_hhl.estimate(problem=problem, num_clock_qubits=k, max_eigenvalue=max_eigenvalue)
print(enhanced_hybrid_hhl_result)

## Compute the control function

We will derive the control function using the solution of the matrix Riccati equation.

In [5]:
# Compute the control function
def compute_control_function(P, B, R, x):
    return -np.linalg.inv(R) @ B.T @ P @ x

P = enhanced_hybrid_hhl_result['P']
u = compute_control_function(P, B_matrix, R_matrix, x_0)
print('Control function:', u)